# BÀI TẬP NUMPY

## Đề Bài

Thực hiện các yêu cầu sau với dữ liệu 'vn_housing_dataset.csv':
1. Lấy ra thông tin từ các cột diện tích, chiều dài, chiều rộng, giá nhà.
2. Kiểm tra các thông tin bất thường, loại bỏ nếu thấy bất hợp lý.
3. Chuẩn hóa thông tin (ví dụ: giá nhà với đơn vị "tỷ" thì đổi hết sang đơn vị "triệu", nếu có).
4. Lấy thêm thông tin từ cột ngày, format lại thành "Ngày_Tháng_Năm"
5. Lưu lại dữ liệu đã làm sạch, chuẩn hóa, loại bỏ bất thường… vào file csv khác

## Script

### Using

In [1]:
import numpy as np
import pandas as pd
from pandas import Timestamp

### Contract Fields

In [2]:
US_L = 'm'  # standard unit of length
US_S = 'm²'  # standard unit of area
US_PR = 'triệu/m²'  # standard unit of price
UL_PR = 'tỷ/m²'  # large unit of length

In [3]:
CN_S = 'Diện tích'  # area column name
CN_L = 'Dài'  # length column name
CN_W = 'Rộng'  # width column name
CN_PR = 'Giá'  # price column name
CN_PR_SRC = 'Giá/m2'  # price column name (source)
CN_DT = 'Ngày'  # price column name

In [4]:
FMT_DT_SRC = '%Y-%m-%d'  # format datime source
FMT_DT_DST = '%d_%m_%Y'  # format datime destination

### Contract Methods

In [5]:
# String split
def s_splt(s):
    return s.str.split()

In [6]:
# String len
def s_len(s):
    return s.str.len()

In [7]:
# String lower
def s_low(s):
    return s.str.lower()

In [8]:
# Header
def hdr(s):
    return s.split()[0]

In [9]:
# Footer
def ftr(s):
    return s.split()[-1]

In [10]:
# Connector
def con(s):
    return s.str.join(' ')

In [11]:
# Refactor
def refactor(s):
    return con(s_splt(s.str.strip()))

In [12]:
# Try parse to number
def try_prs_num(s):
    return pd.to_numeric(s.str[0], errors='coerce')

In [13]:
# Try parse to datetime
def try_prs_dt(s):
    return pd.to_datetime(s, format=FMT_DT_SRC, errors='coerce')

In [14]:
# Try parse value
def try_prs_val(s):
    return s_splt(s).str[0].astype(float)

In [15]:
# Round value and add standard unit
def rnd_unit(val, unit):
    return val.round(2).astype(str) + f' {unit}'

In [16]:
# Round value and add standard unit advanced
def rnd_unit_adv(val, unit):
    return s_splt(val).str[0].astype(float).round(2).astype(str).apply(lambda val: f'{val} {unit}')

### Processing

In [17]:
df = pd.read_csv('vn_housing_dataset.csv')

In [18]:
# Request 1
length = refactor(df[CN_L])
width = refactor(df[CN_W])
area = refactor(df[CN_S])
price = refactor(df[CN_PR_SRC])

In [19]:
# Request 2
length_split = s_splt(length)
width_split = s_splt(width)
area_split = s_splt(area)
price_split = s_splt(price)

length_value = try_prs_num(length_split)
width_value = try_prs_num(width_split)
area_value = try_prs_num(area_split)
price_value = try_prs_num(price_split)

valids = (length_value.notna()
              & width_value.notna()
              & area_value.notna()
              & price_value.notna()
              & length_value.gt(0)
              & width_value.gt(0)
              & area_value.gt(0)
              & price_value.gt(0)
              & s_len(length_split).gt(1)
              & s_len(width_split).gt(1)
              & s_len(area_split).gt(1)
              & s_len(price_split).gt(1)
              & (np.ceil(length_value) * np.ceil(width_value)).ge(np.floor(area_value)))
df_valid = df.loc[valids]

length_valid = rnd_unit(length_value.loc[valids], US_L)
width_valid = rnd_unit(width_value.loc[valids], US_L)
area_valid = rnd_unit(area_value.loc[valids], US_S)
price_valid = df[CN_PR_SRC].loc[valids]

In [20]:
# Request 3
price_valid = rnd_unit_adv(con(s_low(price_valid).apply(lambda x: [str(float(hdr(x)) * 1000) if ftr(x) == UL_PR else hdr(x), ftr(x)])), US_PR)

In [21]:
# Request 4
df_date = df.loc[valids][CN_DT]
date_value = try_prs_dt(df_date)

valids &= (date_value.notna()) & date_value.lt(Timestamp.today())
invalids = ~valids

length_valid = length_valid.loc[valids]
width_valid = width_valid.loc[valids]
area_valid = area_valid.loc[valids]
price_valid = price_valid.loc[valids]
date_valid = pd.to_datetime(date_value.loc[valids]).dt.strftime(FMT_DT_DST)

In [22]:
# Request 5
pd.concat([date_valid, length_valid, width_valid, area_valid, price_valid], axis=1).to_csv('data_valid.csv')
df.loc[invalids].to_csv('data_invalid.csv')